# SQL 작성

In [1]:
import sys
import os
import pandas as pd
from scipy import io
import cv2
from glob import glob
sys.path.append('../../src')
import time
import DISData as DD

## SQL 연결

In [3]:
doUT = DD.SQL()

host:  192.168.0.201
user:  diadmin
password:  ········


SQL connection done


## query 작성

In [4]:
query1 = (f'''CREATE TABLE IF NOT EXISTS DIS_300W_LP_Filp_landmark_info(
            id INT PRIMARY KEY AUTO_INCREMENT,
            img_name VARCHAR(500) NOT NULL,
            img_dir VARCHAR(1000) NOT NULL,
            img_format VARCHAR(10) NOT NULL,
            img_width INT NOT NULL,
            img_height INT NOT NULL,
            color_space VARCHAR(10) NOT NULL,
            label_name VARCHAR(500) NOT NULL,
            label_dir VARCHAR(1000) NOT NULL,
            label_format VARCHAR(10) NOT NULL,
            label VARCHAR(100) NOT NULL,
            landmark VARCHAR(1000) NOT NULL);''')

In [5]:
query2 = (f'''INSERT INTO DIS_300W_LP_Filp_landmark_info 
        (img_name, img_dir, img_format, img_width, img_height, 
        color_space, label_name, label_dir, label_format, label, landmark)
        VALUES (%s, %s, %s, %s, %s,  %s, %s, %s, %s, %s,  %s)''')

## qurey 실행

In [6]:
try:
    doUT.create_table(query1)
except Exception as e:
    print(f"Error during table creation: {e}")

create table done


In [7]:
# label 경로
label_path = 'Z:\\DIS_300W_LP_flip\\data\\RGB\\labeldata'
image_path = 'Z:\\DIS_300W_LP_flip\\data\\RGB\\rawdata'

value_list1 = []
error_list = []
count = 0

label_lists = os.listdir(label_path)
for num, label_list in enumerate(label_lists):
    print(num)
    total_cnt = len(label_lists)
    label_files = os.listdir(os.path.join(label_path,label_list))
    for label_file in label_files:
        label = os.path.join(label_path,label_list,label_file)
        # print(label)
        text = io.loadmat(label)
        # print(text)
        image = label.replace('labeldata','rawdata').replace('.mat','.jpg')    
        img_dir = image
        img_name, img_format = os.path.splitext(os.path.basename(image))
        img_format = img_format.replace('.','')
        image = cv2.imread(img_dir)
        if image is None:
            print(img_dir)
            error_list.append(img_dir)
            continue
        img_width = image.shape[1]
        img_height = image.shape[0]
        color_check = img_dir.split('\\')[-2]

        color_space = 'RGB'
        label_dir = label
        label_name, label_format = os.path.splitext(os.path.basename(label))
        label_format = label_format.replace('.','')
        label = 'face_68'

        pt2d_x = text['pt2d'][0]
        pt2d_y = text['pt2d'][1]

        point = []
        
        for pt in range(len(pt2d_x)):
            point.append((int(float(pt2d_x[pt])),int(float(pt2d_y[pt]))))

        landmark = ','.join(map(str, [ (x,y) for x,y in point]))

        img_dir = img_dir.replace('Z:', 'DataBase').replace('\\','/')
        label_dir = label_dir.replace('Z:', 'DataBase').replace('\\','/')

        value1 = (img_name, img_dir, img_format, img_width, img_height, 
        color_space, label_name, label_dir, label_format, label, landmark)
        value_list1.append(value1)
            
        if len(value_list1) >= 500:
            print(value_list1[0])
            print(f'{num}/{total_cnt}')
            doUT.insert_dataset_values(query2, value_list1)
            value_list1 = []

doUT.insert_dataset_values(query2, value_list1)
# print(value_list2)
value_list1 = []
print('done')      


0
('AFW_1051618982_1_0', 'DataBase/DIS_300W_LP_flip/data/RGB/rawdata/AFW_Flip/AFW_1051618982_1_0.jpg', 'jpg', 450, 450, 'RGB', 'AFW_1051618982_1_0', 'DataBase/DIS_300W_LP_flip/data/RGB/labeldata/AFW_Flip/AFW_1051618982_1_0.mat', 'mat', 'face_68', '(313, 242),(312, 266),(309, 291),(306, 314),(297, 336),(281, 353),(265, 369),(247, 380),(227, 384),(204, 381),(183, 370),(164, 354),(150, 335),(142, 314),(138, 293),(136, 272),(134, 249),(298, 228),(285, 218),(270, 215),(254, 217),(240, 222),(205, 223),(189, 218),(173, 216),(157, 218),(146, 228),(223, 243),(224, 257),(225, 272),(225, 287),(244, 296),(235, 297),(226, 301),(216, 298),(208, 296),(281, 246),(271, 239),(259, 239),(248, 249),(259, 249),(272, 249),(199, 251),(187, 240),(175, 241),(164, 249),(174, 252),(187, 251),(264, 318),(248, 315),(235, 313),(227, 315),(217, 313),(203, 316),(188, 320),(203, 333),(218, 334),(228, 334),(236, 334),(250, 330),(258, 320),(236, 323),(227, 323),(218, 324),(193, 322),(218, 324),(227, 323),(236, 323)')
0/

In [7]:
doUT.connect_end()

SQL connection closed
